# HW-3
### Q1 GDA

In [10]:
import numpy as np
from scipy.stats import false_discovery_control
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [11]:
import warnings

warnings.filterwarnings('ignore', message='divide by zero encountered in det')
warnings.filterwarnings('ignore', message='overflow encountered in det')
warnings.filterwarnings('ignore', message='invalid value encountered in det')
warnings.filterwarnings('ignore', category=RuntimeWarning)

In [12]:
data = np.loadtxt("spambase/spambase.data", delimiter=",")
X = data[:, :-1]
y = data[:, -1].astype(int)

scaler = StandardScaler()
X_norm = scaler.fit_transform(X)

In [13]:
class GDA:
    def __init__(self, shared_covariance = False):
        self.shared_covariance = shared_covariance
        self.priors = {}
        self.means = {}
        self.covariances = {}
        self.shared_cov = None

    def fit(self, X, y):
        self.classes = np.unique(y)
        n_samples = X.shape[0]
        n_features = X.shape[1]

        for c in self.classes:
            X_c = X[y==c]

            # prior:
            self.priors[c] = len(X_c) / n_samples

            # mean vector:
            self.means[c] = np.mean(X_c, axis=0)

            # cov for QDA
            if not self.shared_covariance:
                self.covariances[c] = np.cov(X_c.T, bias=False)

                # reg param
                self.covariances[c] += 1e-6 * np.eye(n_features)

        # single shared cov for LDA:
        if self.shared_covariance:
            self.shared_cov = np.cov(X.T, bias=False)

            # reg param:
            self.shared_cov += 1e-6 * np.eye(n_features)

    def predict(self, X):
        """predicting class for each instance using log prob"""

        predictions = []

        for x in X:
            log_probs = {}

            for c in self.classes:
                if self.shared_covariance:
                    cov = self.shared_cov
                else:
                    cov = self.covariances[c]

                mean_c = self.means[c]
                diff = x - mean_c
                sign, logdet = np.linalg.slogdet(cov)
                
                cov_inv = np.linalg.inv(cov)

                # log P(x|y=c) = -0.5 * [d*log(2π) + log|Σ| + (x-μ)^T Σ^(-1) (x-μ)]
                d = len(x)
                LL = -0.5 * (d * np.log(2*np.pi) + logdet + diff @ cov_inv @ diff)

                # log prior
                log_prior = np.log(self.priors[c])

                # log posterior
                log_probs[c] = log_prior + LL

            # predicting classes w/ max log prob
            predictions.append(max(log_probs, key=log_probs.get))
        return np.array(predictions)

k-fold cross validation:

In [14]:
k_folds = 10
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

lda_results = {'train_acc': [], 'test_acc': []}
qda_results = {'train_acc': [], 'test_acc': []}

for fold, (train_idx, test_idx) in enumerate(kf.split(X_norm), 1):
    X_train, X_test = X_norm[train_idx], X_norm[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    # Test LDA (shared covariance)
    lda = GDA(shared_covariance=True)
    lda.fit(X_train, y_train)

    train_pred_lda = lda.predict(X_train)
    test_pred_lda = lda.predict(X_test)

    lda_train_acc = accuracy_score(y_train, train_pred_lda)
    lda_test_acc = accuracy_score(y_test, test_pred_lda)

    lda_results['train_acc'].append(lda_train_acc)
    lda_results['test_acc'].append(lda_test_acc)

    # Test QDA (class-specific covariances)
    qda = GDA(shared_covariance=False)
    qda.fit(X_train, y_train)

    train_pred_qda = qda.predict(X_train)
    test_pred_qda = qda.predict(X_test)

    qda_train_acc = accuracy_score(y_train, train_pred_qda)
    qda_test_acc = accuracy_score(y_test, test_pred_qda)

    qda_results['train_acc'].append(qda_train_acc)
    qda_results['test_acc'].append(qda_test_acc)

    print(f"Fold {fold}: LDA Test={lda_test_acc:.4f}, QDA Test={qda_test_acc:.4f}")

Fold 1: LDA Test=0.8503, QDA Test=0.8395
Fold 2: LDA Test=0.8696, QDA Test=0.8370
Fold 3: LDA Test=0.8674, QDA Test=0.8478
Fold 4: LDA Test=0.8674, QDA Test=0.8022
Fold 5: LDA Test=0.8587, QDA Test=0.7978
Fold 6: LDA Test=0.8652, QDA Test=0.8283
Fold 7: LDA Test=0.8348, QDA Test=0.8478
Fold 8: LDA Test=0.9022, QDA Test=0.8217
Fold 9: LDA Test=0.8543, QDA Test=0.8478
Fold 10: LDA Test=0.8696, QDA Test=0.8283


In [15]:
# Calculate and display the final results
print("\n" + "="*50)
print("FINAL RESULTS (averaged across 10 folds):")
print("="*50)

print("\nLDA (Single Shared Covariance):")
print(f"  Train Accuracy: {np.mean(lda_results['train_acc']):.4f} ± {np.std(lda_results['train_acc']):.4f}")
print(f"  Test Accuracy:  {np.mean(lda_results['test_acc']):.4f} ± {np.std(lda_results['test_acc']):.4f}")

print("\nQDA (Class-Specific Covariances):")
print(f"  Train Accuracy: {np.mean(qda_results['train_acc']):.4f} ± {np.std(qda_results['train_acc']):.4f}")
print(f"  Test Accuracy:  {np.mean(qda_results['test_acc']):.4f} ± {np.std(qda_results['test_acc']):.4f}")


FINAL RESULTS (averaged across 10 folds):

LDA (Single Shared Covariance):
  Train Accuracy: 0.8650 ± 0.0017
  Test Accuracy:  0.8639 ± 0.0164

QDA (Class-Specific Covariances):
  Train Accuracy: 0.8326 ± 0.0041
  Test Accuracy:  0.8298 ± 0.0172
